# ECOS 경제지표 예측 모델
LSTM을 활용한 6개 핵심 경제지표 예측

In [12]:
pip install xgboost pandas numpy matplotlib seaborn scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [14]:
# XGBoost 기반 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
import xgboost as xgb
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [15]:
# ecos_final_data.csv 파일 로드
import os

# 파일 경로 설정
current_dir = os.path.dirname(os.path.abspath('ecos_predict.ipynb'))
data_path = os.path.join(current_dir, 'ecos_final_data.csv')

try:
    # 데이터 로드
    df = pd.read_csv(data_path, encoding='utf-8-sig')
    print(f"데이터 로드 완료: {df.shape}")
    print(f"컬럼 수: {len(df.columns)}")
    print(f"기간: {df.index[0]} ~ {df.index[-1]}")
    
    # 데이터 기본 정보 확인
    print("\n데이터 기본 정보:")
    print(df.info())
    
    print("\n데이터 미리보기:")
    print(df.head())
    
except FileNotFoundError:
    print(f"파일을 찾을 수 없습니다: {data_path}")
    print("현재 디렉토리의 파일 목록:")
    files = [f for f in os.listdir(current_dir) if f.endswith('.csv')]
    for file in files:
        print(f"  - {file}")

데이터 로드 완료: (38, 26)
컬럼 수: 26
기간: 0 ~ 37

데이터 기본 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 26 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   year                                38 non-null     int64  
 1   quarter                             38 non-null     object 
 2   base_rate                           38 non-null     float64
 3   base_rate_qdiff_bp                  38 non-null     float64
 4   cpi                                 38 non-null     float64
 5   cpi_qoq                             38 non-null     float64
 6   exchange_usd_krw_close              38 non-null     float64
 7   exchange_qstd                       38 non-null     float64
 8   market_rate_treasury_bond_3yr       38 non-null     float64
 9   market_rate_treasury_bond_10yr      38 non-null     float64
 10  term_spread                         38 non-null     float64


In [36]:
# 예측 대상 지표 선택
# 6개 핵심 경제지표 정의
target_columns = [
    'leading_index',           # 가장 포괄적인 공식 선행지표
    'term_spread',            # 경기 전환점 예측 지표
    'credit_spread',          # 금융 스트레스와 신용 여건 지표
    'esi',                    # 포괄적인 민간 부문 심리 지표
    'exchange_usd_krw_close', # 대외 부문 영향 지표
    'cpi_qoq'                 # 물가 상승 동력 지표
]

# 선택된 컬럼 존재 여부 확인
available_columns = [col for col in target_columns if col in df.columns]
missing_columns = [col for col in target_columns if col not in df.columns]

print(f"사용 가능한 지표 ({len(available_columns)}개): {available_columns}")
if missing_columns:
    print(f"누락된 지표 ({len(missing_columns)}개): {missing_columns}")

# 예측용 데이터셋 생성
if available_columns:
    data = df[available_columns].copy()
    print(f"\n예측용 데이터셋 생성: {data.shape}")
    
    # 결측값 확인
    missing_counts = data.isnull().sum()
    if missing_counts.sum() > 0:
        print("\n결측값 현황:")
        print(missing_counts[missing_counts > 0])
        
        # 결측값 처리 (선형 보간)
        data = data.interpolate(method='linear').fillna(method='bfill').fillna(method='ffill')
        print("결측값 선형 보간 처리 완료")
    else:
        print("결측값 없음")
    
    # 기초 통계량 확인
    print("\n기초 통계량:")
    print(data.describe())
else:
    print("예측 가능한 지표가 없습니다.")

사용 가능한 지표 (6개): ['leading_index', 'term_spread', 'credit_spread', 'esi', 'exchange_usd_krw_close', 'cpi_qoq']

예측용 데이터셋 생성: (38, 6)
결측값 없음

기초 통계량:
       leading_index  term_spread  credit_spread         esi  \
count      38.000000    38.000000      38.000000   38.000000   
mean      100.342982     0.320386       6.131482   95.241228   
std         1.092753     0.221627       0.197528    7.894532   
min        98.366667    -0.027333       5.761667   64.333333   
25%        99.275000     0.141417       5.991083   92.691667   
50%       100.433333     0.320167       6.145333   95.416667   
75%       101.116667     0.476583       6.278000   99.775000   
max       102.666667     0.900333       6.414333  107.333333   

       exchange_usd_krw_close    cpi_qoq  
count               38.000000  38.000000  
mean              1216.517719   0.537241  
std                104.135903   0.529426  
min               1072.816667  -0.513008  
25%               1128.231667   0.211753  
50%              

In [ ]:
# R² 향상을 위한 간단하고 효과적인 Feature Engineering
def create_simple_effective_features(data, max_lag=2):
    """
    R² 성능 향상을 위한 간단하고 효과적인 Feature 생성
    - 샘플 수 대비 적절한 Feature 수 유지
    - 노이즈 최소화, 핵심 패턴 포착
    """
    result_data = data.copy()
    
    print("간단하고 효과적인 Feature Engineering...")
    
    # 각 지표별로 핵심 feature만 생성
    for col in data.columns:
        # 1. 핵심 Lag features (1~2분기)
        for lag in range(1, max_lag + 1):
            result_data[f'{col}_lag_{lag}'] = data[col].shift(lag)
        
        # 2. 4분기 이동평균 (트렌드)
        result_data[f'{col}_ma4'] = data[col].rolling(window=4, min_periods=1).mean()
        
        # 3. 분기 변화율 (모멘텀)
        result_data[f'{col}_qoq'] = data[col].pct_change()
        
        # 4. 1차 차분 (변화량)
        result_data[f'{col}_diff'] = data[col].diff()
        
        # 5. 4분기 대비 상대 위치 (0~1)
        rolling_min = data[col].rolling(window=4, min_periods=1).min()
        rolling_max = data[col].rolling(window=4, min_periods=1).max()
        result_data[f'{col}_pos'] = (data[col] - rolling_min) / (rolling_max - rolling_min + 1e-8)
    
    # 6. 지표 간 상호작용 (주요 조합만)
    if len(data.columns) >= 3:
        # 경기선행지수 vs 다른 지표 (2개만)
        base_col = data.columns[0]  # leading_index
        for col in data.columns[1:3]:  # term_spread, credit_spread
            result_data[f'{base_col}_{col}_ratio'] = data[base_col] / (data[col] + 1e-8)
    
    return result_data

# Feature 중요도 기반 선택 (개선)
def select_top_features(X, y, target_col, max_features=15):
    """
    통계적 중요도와 트리 기반 중요도를 결합한 feature 선택
    """
    from sklearn.feature_selection import SelectKBest, f_regression
    from sklearn.ensemble import RandomForestRegressor
    
    # 데이터 정리
    X_clean = X.replace([np.inf, -np.inf], np.nan).fillna(0)
    
    # 1차: F-test 기반 선택 (상위 30개)
    selector = SelectKBest(score_func=f_regression, k=min(30, X_clean.shape[1]))
    X_selected = selector.fit_transform(X_clean, y[target_col])
    selected_features = X.columns[selector.get_support()]
    
    # 2차: Random Forest 중요도 (상위 max_features개)
    rf = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)
    rf.fit(X_clean[selected_features], y[target_col])
    
    importance_scores = rf.feature_importances_
    top_indices = np.argsort(importance_scores)[-max_features:]
    final_features = selected_features[top_indices]
    
    return X_clean[final_features], final_features

# 간단한 Feature engineering 수행
print("R² 향상을 위한 간단하고 효과적인 Feature Engineering 시작...")
feature_data = create_simple_effective_features(data, max_lag=2)
print(f"Feature Engineering 완료: {feature_data.shape}")

# 결측값 처리
feature_data = feature_data.dropna()
feature_data = feature_data.replace([np.inf, -np.inf], np.nan).fillna(0)
print(f"결측값 처리 후: {feature_data.shape}")

# target과 feature 분리
target_features = [col for col in feature_data.columns if col in available_columns]
engineered_features = [col for col in feature_data.columns if col not in available_columns]

X_features = feature_data[engineered_features]
y_targets = feature_data[target_features]

print(f"\n초기 Feature 개수: {len(engineered_features)}")

# 전체적으로 중요한 feature들 선택
print(f"\n전체적으로 중요한 Feature 선택...")

# 모든 target에 대해 중요한 feature 수집
all_important_features = set()
for target in available_columns:
    X_selected, selected_features = select_top_features(X_features, y_targets, target, max_features=12)
    all_important_features.update(selected_features)
    print(f"  {target}: {len(selected_features)}개 features 선택")

# 최종 feature 집합 (최대 20개로 제한)
final_features = list(all_important_features)[:20]
print(f"\n최종 선택된 features: {len(final_features)}개")

# 최종 데이터셋 구성
X_final = X_features[final_features]
y_final = y_targets

# Feature scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_final)
X_scaled = pd.DataFrame(X_scaled, columns=final_features, index=X_final.index)

# 시계열 분할 (더 많은 훈련 데이터 - 85%)
TRAIN_RATIO = 0.85
train_size = int(len(X_scaled) * TRAIN_RATIO)

X_train = X_scaled[:train_size]
X_test = X_scaled[train_size:]
y_train = y_final[:train_size]
y_test = y_final[train_size:]

print(f"\n최종 데이터 구성:")
print(f"  훈련 데이터: X_train {X_train.shape}, y_train {y_train.shape}")
print(f"  테스트 데이터: X_test {X_test.shape}, y_test {y_test.shape}")
print(f"  Feature/Sample 비율: {X_train.shape[1]/X_train.shape[0]:.2f}")

if X_train.shape[1] <= X_train.shape[0] * 0.8:
    print("  ✓ 과적합 위험 낮음 (Feature < 0.8 * Sample)")
elif X_train.shape[1] <= X_train.shape[0]:
    print("  ⚠️ 과적합 위험 중간 (Feature ≈ Sample)")
else:
    print("  ❌ 과적합 위험 높음 (Feature > Sample)")

print(f"\n데이터 품질:")
print(f"  NaN: {np.isnan(X_train).sum().sum()}")
print(f"  Inf: {np.isinf(X_train).sum().sum()}")
print(f"  훈련 샘플 충분성: {'충분' if len(X_train) >= 25 else '부족'}")

# 선택된 feature 유형 분석
feature_types = {
    'lag': len([f for f in final_features if 'lag_' in f]),
    'ma': len([f for f in final_features if '_ma4' in f]),
    'change': len([f for f in final_features if '_qoq' in f or '_diff' in f]),
    'position': len([f for f in final_features if '_pos' in f]),
    'interaction': len([f for f in final_features if '_ratio' in f]),
    'original': len([f for f in final_features if f in available_columns])
}

print(f"\n선택된 Feature 유형별 분포:")
for ftype, count in feature_types.items():
    if count > 0:
        print(f"  {ftype}: {count}개")

print("=" * 60)

R² 성능 향상을 위한 고도화된 Feature Engineering 시작...
R² 향상을 위한 고도화된 Feature Engineering...
Feature Engineering 완료: (38, 147)
결측값 처리 후: (34, 147)

초기 Feature 분리:
  생성된 features: 141
  Target features: 6

Target별 최적 Feature 선택...
  leading_index: 25개 features 선택
  term_spread: 25개 features 선택
  credit_spread: 25개 features 선택
  leading_index: 25개 features 선택
  term_spread: 25개 features 선택
  credit_spread: 25개 features 선택
  esi: 25개 features 선택
  exchange_usd_krw_close: 25개 features 선택
  esi: 25개 features 선택
  exchange_usd_krw_close: 25개 features 선택
  cpi_qoq: 25개 features 선택

최종 선택된 features: 54개

최종 데이터 구성:
  훈련 데이터: X_train (27, 54), y_train (27, 6)
  테스트 데이터: X_test (7, 54), y_test (7, 6)
  Feature/Sample 비율: 2.00
  ⚠️ 과적합 위험 있음
  데이터 품질: NaN=0, Inf=0
  cpi_qoq: 25개 features 선택

최종 선택된 features: 54개

최종 데이터 구성:
  훈련 데이터: X_train (27, 54), y_train (27, 6)
  테스트 데이터: X_test (7, 54), y_test (7, 6)
  Feature/Sample 비율: 2.00
  ⚠️ 과적합 위험 있음
  데이터 품질: NaN=0, Inf=0


In [ ]:
# R² 향상을 위한 앙상블 XGBoost 모델
def train_ensemble_xgboost_models(X_train, y_train, X_test, y_test, target_columns, n_models=3):
    """
    R² 성능 향상을 위한 앙상블 XGBoost 모델
    - 여러 모델의 앙상블로 안정성과 성능 향상
    - 각 모델은 다른 하이퍼파라미터와 랜덤 시드 사용
    """
    from sklearn.metrics import r2_score as sklearn_r2_score
    
    models = {}
    predictions = {}
    
    # 다양한 하이퍼파라미터 설정
    param_sets = [
        {  # 모델 1: 깊이 중심
            'n_estimators': 200,
            'max_depth': 6,
            'learning_rate': 0.05,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'reg_alpha': 0.1,
            'reg_lambda': 0.1,
            'min_child_weight': 3,
            'gamma': 0.1,
            'random_state': 42
        },
        {  # 모델 2: 넓이 중심
            'n_estimators': 300,
            'max_depth': 4,
            'learning_rate': 0.08,
            'subsample': 0.7,
            'colsample_bytree': 0.9,
            'reg_alpha': 0.2,
            'reg_lambda': 0.05,
            'min_child_weight': 5,
            'gamma': 0.2,
            'random_state': 123
        },
        {  # 모델 3: 균형 중심
            'n_estimators': 250,
            'max_depth': 5,
            'learning_rate': 0.06,
            'subsample': 0.75,
            'colsample_bytree': 0.85,
            'reg_alpha': 0.15,
            'reg_lambda': 0.15,
            'min_child_weight': 4,
            'gamma': 0.15,
            'random_state': 456
        }
    ]
    
    print("R² 향상을 위한 앙상블 XGBoost 모델 훈련...")
    print("=" * 70)
    
    for target in target_columns:
        print(f"\n[{target}] 앙상블 모델 훈련 중...")
        
        target_models = []
        target_predictions = []
        ensemble_train_preds = []
        ensemble_test_preds = []
        
        # 각 파라미터 세트로 모델 훈련
        for i, params in enumerate(param_sets[:n_models]):
            print(f"  모델 {i+1}/{n_models} 훈련 중...")
            
            # 모델 생성
            model = XGBRegressor(
                **params,
                n_jobs=-1,
                verbosity=0
            )
            
            # 훈련
            try:
                from xgboost.callback import EarlyStopping
                model.fit(
                    X_train, y_train[target],
                    eval_set=[(X_test, y_test[target])],
                    callbacks=[EarlyStopping(rounds=30)],
                    verbose=False
                )
            except:
                model.fit(X_train, y_train[target])
            
            # 예측
            train_pred = model.predict(X_train)
            test_pred = model.predict(X_test)
            
            target_models.append(model)
            ensemble_train_preds.append(train_pred)
            ensemble_test_preds.append(test_pred)
            
            # 개별 모델 성능
            r2 = sklearn_r2_score(y_test[target], test_pred)
            print(f"    모델 {i+1} R²: {r2:.4f}")
        
        # 앙상블 예측 (평균)
        final_train_pred = np.mean(ensemble_train_preds, axis=0)
        final_test_pred = np.mean(ensemble_test_preds, axis=0)
        
        # 앙상블 성능
        ensemble_train_r2 = sklearn_r2_score(y_train[target], final_train_pred)
        ensemble_test_r2 = sklearn_r2_score(y_test[target], final_test_pred)
        
        print(f"  앙상블 R² - 훈련: {ensemble_train_r2:.4f}, 테스트: {ensemble_test_r2:.4f}")
        
        # 저장
        models[target] = {
            'individual_models': target_models,
            'ensemble_train_pred': final_train_pred,
            'ensemble_test_pred': final_test_pred
        }
        
        predictions[target] = {
            'train_pred': final_train_pred,
            'test_pred': final_test_pred
        }
    
    print("=" * 70)
    print("앙상블 XGBoost 모델 훈련 완료")
    
    return models, predictions

# 개별 target별 최적화 함수
def optimize_target_specific_params(X_train, y_train, target, cv_folds=5):
    """
    개별 target에 특화된 하이퍼파라미터 최적화
    """
    from sklearn.model_selection import RandomizedSearchCV
    from scipy.stats import randint, uniform
    
    print(f"  {target} 전용 하이퍼파라미터 최적화...")
    
    # target별 특화 파라미터 분포
    param_distributions = {
        'n_estimators': randint(100, 400),
        'max_depth': randint(3, 8),
        'learning_rate': uniform(0.01, 0.15),
        'subsample': uniform(0.6, 0.3),
        'colsample_bytree': uniform(0.6, 0.3),
        'reg_alpha': uniform(0.01, 0.5),
        'reg_lambda': uniform(0.01, 0.5),
        'min_child_weight': randint(1, 10),
        'gamma': uniform(0.01, 0.5)
    }
    
    base_model = XGBRegressor(
        random_state=42,
        n_jobs=-1,
        verbosity=0
    )
    
    # 랜덤 서치
    random_search = RandomizedSearchCV(
        estimator=base_model,
        param_distributions=param_distributions,
        n_iter=30,  # 30회 시도
        cv=cv_folds,
        scoring='r2',
        n_jobs=-1,
        verbose=0,
        random_state=42
    )
    
    random_search.fit(X_train, y_train[target])
    
    return random_search.best_params_, random_search.best_score_

# 최적화된 모델 훈련 함수
def train_optimized_ensemble_models(X_train, y_train, X_test, y_test, target_columns):
    """
    각 target별로 최적화된 앙상블 모델 훈련
    """
    print("Target별 최적화된 앙상블 모델 훈련...")
    print("=" * 70)
    
    optimized_models = {}
    optimized_predictions = {}
    
    for target in target_columns:
        print(f"\n[{target}] 최적화 중...")
        
        # target별 최적 파라미터 찾기
        best_params, best_cv_score = optimize_target_specific_params(X_train, y_train, target)
        print(f"  최적 CV R²: {best_cv_score:.4f}")
        
        # 최적 파라미터로 앙상블 모델 훈련
        ensemble_models, ensemble_preds = train_ensemble_xgboost_models(
            X_train, y_train, X_test, y_test, [target], n_models=3
        )
        
        optimized_models[target] = ensemble_models[target]
        optimized_predictions[target] = ensemble_preds[target]
    
    return optimized_models, optimized_predictions

In [59]:
# R² 향상을 위한 간단하고 효과적인 하이퍼파라미터 최적화
def effective_hyperparameter_optimization(X_train, y_train, target_columns):
    """
    R² 성능 최적화를 위한 실용적인 하이퍼파라미터 튜닝
    """
    from sklearn.model_selection import GridSearchCV
    
    print("R² 향상을 위한 효과적인 하이퍼파라미터 최적화...")
    print("=" * 70)
    
    optimized_params_dict = {}
    optimization_results = {}
    
    # 실용적인 하이퍼파라미터 그리드 (Early stopping 제거)
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 4, 5, 6],
        'learning_rate': [0.01, 0.05, 0.1, 0.15],
        'subsample': [0.7, 0.8, 0.9],
        'colsample_bytree': [0.7, 0.8, 0.9],
        'reg_alpha': [0.0, 0.1, 0.5],
        'reg_lambda': [0.0, 0.1, 0.5]
    }
    
    for target in target_columns:
        print(f"\n[{target}] 하이퍼파라미터 최적화...")
        
        # Early stopping 없는 기본 모델
        base_model = XGBRegressor(
            random_state=42,
            n_jobs=-1,
            verbosity=0
            # early_stopping_rounds 제거
        )
        
        # 그리드 서치 (작은 그리드로 시작)
        grid_search = GridSearchCV(
            estimator=base_model,
            param_grid={
                'n_estimators': [100, 200],
                'max_depth': [3, 4, 5],
                'learning_rate': [0.05, 0.1],
                'subsample': [0.8, 0.9],
                'colsample_bytree': [0.8, 0.9],
                'reg_alpha': [0.1, 0.5],
                'reg_lambda': [0.1, 0.5]
            },
            cv=3,  # 3-fold CV
            scoring='r2',
            n_jobs=-1,
            verbose=0
        )
        
        # 최적화 실행
        grid_search.fit(X_train, y_train[target])
        
        best_params = grid_search.best_params_
        best_score = grid_search.best_score_
        
        # 결과 저장
        optimized_params_dict[target] = best_params
        optimization_results[target] = {
            'best_cv_r2': best_score,
            'best_params': best_params
        }
        
        print(f"  최적 CV R²: {best_score:.4f}")
        print(f"  주요 파라미터: lr={best_params['learning_rate']:.3f}, "
              f"depth={best_params['max_depth']}, "
              f"n_est={best_params['n_estimators']}")
    
    # 전체 최적화 결과 요약
    print(f"\n하이퍼파라미터 최적화 완료:")
    print("=" * 70)
    avg_cv_r2 = np.mean([result['best_cv_r2'] for result in optimization_results.values()])
    print(f"평균 CV R²: {avg_cv_r2:.4f}")
    
    # 최고 성능 target 식별
    best_target = max(optimization_results.keys(), 
                     key=lambda k: optimization_results[k]['best_cv_r2'])
    best_r2 = optimization_results[best_target]['best_cv_r2']
    print(f"최고 성능: {best_target} (R² = {best_r2:.4f})")
    
    return optimized_params_dict, optimization_results

# 최적화 실행
optimized_params_dict, optimization_results = effective_hyperparameter_optimization(
    X_train, y_train, available_columns
)

# 최적화 결과 분석
print(f"\n최적화 결과 상세 분석:")
print("=" * 50)

for target, result in optimization_results.items():
    params = result['best_params']
    cv_r2 = result['best_cv_r2']
    
    print(f"\n{target} (CV R²: {cv_r2:.4f}):")
    print(f"  학습률: {params['learning_rate']:.4f}")
    print(f"  트리 깊이: {params['max_depth']}")
    print(f"  트리 개수: {params['n_estimators']}")
    print(f"  정규화: α={params['reg_alpha']:.3f}, λ={params['reg_lambda']:.3f}")
    
    # 성능 등급
    if cv_r2 > 0.7:
        grade = "우수"
    elif cv_r2 > 0.5:
        grade = "양호"
    elif cv_r2 > 0.3:
        grade = "보통"
    else:
        grade = "개선필요"
    
    print(f"  성능 등급: {grade}")

print("=" * 50)

R² 향상을 위한 효과적인 하이퍼파라미터 최적화...

[leading_index] 하이퍼파라미터 최적화...
  최적 CV R²: 0.7355
  주요 파라미터: lr=0.050, depth=5, n_est=200

[term_spread] 하이퍼파라미터 최적화...
  최적 CV R²: 0.7355
  주요 파라미터: lr=0.050, depth=5, n_est=200

[term_spread] 하이퍼파라미터 최적화...
  최적 CV R²: 0.0942
  주요 파라미터: lr=0.100, depth=3, n_est=200

[credit_spread] 하이퍼파라미터 최적화...
  최적 CV R²: 0.0942
  주요 파라미터: lr=0.100, depth=3, n_est=200

[credit_spread] 하이퍼파라미터 최적화...
  최적 CV R²: 0.1047
  주요 파라미터: lr=0.100, depth=4, n_est=100

[esi] 하이퍼파라미터 최적화...
  최적 CV R²: 0.1047
  주요 파라미터: lr=0.100, depth=4, n_est=100

[esi] 하이퍼파라미터 최적화...
  최적 CV R²: 0.3953
  주요 파라미터: lr=0.100, depth=5, n_est=200

[exchange_usd_krw_close] 하이퍼파라미터 최적화...
  최적 CV R²: 0.3953
  주요 파라미터: lr=0.100, depth=5, n_est=200

[exchange_usd_krw_close] 하이퍼파라미터 최적화...
  최적 CV R²: -2.8229
  주요 파라미터: lr=0.100, depth=3, n_est=100

[cpi_qoq] 하이퍼파라미터 최적화...
  최적 CV R²: -2.8229
  주요 파라미터: lr=0.100, depth=3, n_est=100

[cpi_qoq] 하이퍼파라미터 최적화...
  최적 CV R²: -0.2545
  주요 파라미터: lr=0.100, dep

In [60]:
# 최적화된 파라미터로 앙상블 모델 훈련
def train_final_optimized_models(X_train, y_train, X_test, y_test, optimized_params_dict, target_columns):
    """
    최적화된 파라미터로 최종 앙상블 모델 훈련
    """
    from sklearn.metrics import r2_score as sklearn_r2_score
    
    models = {}
    predictions = {}
    
    print("최적화된 파라미터로 최종 앙상블 모델 훈련...")
    print("=" * 70)
    
    for target in target_columns:
        print(f"\n[{target}] 최종 앙상블 모델 구성...")
        
        # 최적 파라미터 가져오기
        best_params = optimized_params_dict[target].copy()
        best_params.update({
            'random_state': 42,
            'n_jobs': -1,
            'verbosity': 0
        })
        
        # 3개의 다양한 앙상블 모델 생성
        ensemble_models = []
        ensemble_train_preds = []
        ensemble_test_preds = []
        
        for i in range(3):
            # 각 모델에 약간의 변화 주기
            model_params = best_params.copy()
            model_params['random_state'] = 42 + i * 100
            
            if i == 1:  # 두 번째 모델: 더 보수적
                model_params['learning_rate'] *= 0.8
                model_params['reg_alpha'] *= 1.2
            elif i == 2:  # 세 번째 모델: 더 공격적
                model_params['learning_rate'] *= 1.2
                model_params['max_depth'] = min(model_params['max_depth'] + 1, 10)
            
            # 모델 생성 및 훈련
            model = XGBRegressor(**model_params)
            
            try:
                from xgboost.callback import EarlyStopping
                model.fit(
                    X_train, y_train[target],
                    eval_set=[(X_test, y_test[target])],
                    callbacks=[EarlyStopping(rounds=30)],
                    verbose=False
                )
            except:
                model.fit(X_train, y_train[target])
            
            # 예측
            train_pred = model.predict(X_train)
            test_pred = model.predict(X_test)
            
            ensemble_models.append(model)
            ensemble_train_preds.append(train_pred)
            ensemble_test_preds.append(test_pred)
            
            # 개별 모델 성능
            individual_r2 = sklearn_r2_score(y_test[target], test_pred)
            print(f"  모델 {i+1} R²: {individual_r2:.4f}")
        
        # 앙상블 예측 (가중 평균 - 성능 기반)
        weights = []
        for i, preds in enumerate(ensemble_test_preds):
            r2 = sklearn_r2_score(y_test[target], preds)
            weight = max(0.1, r2)  # 최소 가중치 0.1
            weights.append(weight)
        
        # 정규화
        weights = np.array(weights)
        weights = weights / weights.sum()
        
        # 가중 평균 계산
        final_train_pred = np.average(ensemble_train_preds, axis=0, weights=weights)
        final_test_pred = np.average(ensemble_test_preds, axis=0, weights=weights)
        
        # 최종 성능
        final_train_r2 = sklearn_r2_score(y_train[target], final_train_pred)
        final_test_r2 = sklearn_r2_score(y_test[target], final_test_pred)
        
        print(f"  앙상블 가중치: {weights}")
        print(f"  최종 R² - 훈련: {final_train_r2:.4f}, 테스트: {final_test_r2:.4f}")
        
        # 결과 저장
        models[target] = {
            'ensemble_models': ensemble_models,
            'weights': weights,
            'best_params': best_params
        }
        
        predictions[target] = {
            'train_pred': final_train_pred,
            'test_pred': final_test_pred
        }
    
    print("=" * 70)
    print("최종 최적화된 앙상블 모델 훈련 완료")
    
    return models, predictions

# 최종 모델 훈련 실행
models, predictions = train_final_optimized_models(
    X_train, y_train, X_test, y_test, optimized_params_dict, available_columns
)

# 예측 결과 통합
y_pred_combined = np.zeros((len(X_test), len(available_columns)))
for i, target in enumerate(available_columns):
    y_pred_combined[:, i] = predictions[target]['test_pred']

print(f"\n최종 결과 요약:")
print(f"  - 앙상블 모델 수: {len(models)}")
print(f"  - 예측 결과 shape: {y_pred_combined.shape}")
print(f"  - 각 target별 3개 모델의 가중 앙상블 적용")

# 전체 성능 미리보기
print(f"\n성능 미리보기:")
for target in available_columns:
    from sklearn.metrics import r2_score as sklearn_r2_score
    r2 = sklearn_r2_score(y_test[target], predictions[target]['test_pred'])
    print(f"  {target}: R² = {r2:.4f}")

print("=" * 70)

최적화된 파라미터로 최종 앙상블 모델 훈련...

[leading_index] 최종 앙상블 모델 구성...
  모델 1 R²: -0.2763
  모델 2 R²: -0.3980
  모델 3 R²: 0.1548
  앙상블 가중치: [0.28181912 0.28181912 0.43636177]
  최종 R² - 훈련: 0.9996, 테스트: -0.0901

[term_spread] 최종 앙상블 모델 구성...
  모델 1 R²: 0.3262
  모델 2 R²: -0.3980
  모델 3 R²: 0.1548
  앙상블 가중치: [0.28181912 0.28181912 0.43636177]
  최종 R² - 훈련: 0.9996, 테스트: -0.0901

[term_spread] 최종 앙상블 모델 구성...
  모델 1 R²: 0.3262
  모델 2 R²: 0.3094
  모델 3 R²: -0.0051
  앙상블 가중치: [0.44342556 0.42061873 0.13595571]
  최종 R² - 훈련: 0.8706, 테스트: 0.2840

[credit_spread] 최종 앙상블 모델 구성...
  모델 1 R²: 0.5403
  모델 2 R²: 0.5337
  모델 2 R²: 0.3094
  모델 3 R²: -0.0051
  앙상블 가중치: [0.44342556 0.42061873 0.13595571]
  최종 R² - 훈련: 0.8706, 테스트: 0.2840

[credit_spread] 최종 앙상블 모델 구성...
  모델 1 R²: 0.5403
  모델 2 R²: 0.5337
  모델 3 R²: 0.5819
  앙상블 가중치: [0.32627278 0.32231816 0.35140906]
  최종 R² - 훈련: 0.8608, 테스트: 0.5541

[esi] 최종 앙상블 모델 구성...
  모델 1 R²: -2.1655
  모델 3 R²: 0.5819
  앙상블 가중치: [0.32627278 0.32231816 0.35140906]
  최종 R² - 훈

In [61]:
# XGBoost 모델 평가 및 성능 분석
def evaluate_xgboost_models(y_true, y_pred, models, feature_names):
    """
    XGBoost 모델들의 성능 평가
    """
    # 함수 내에서 명시적 import로 변수명 충돌 방지
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score as sklearn_r2_score
    
    metrics = {}
    
    print("XGBoost 모델 성능 평가")
    print("=" * 80)
    print(f"{'지표명':20} | {'MSE':>8} | {'RMSE':>8} | {'MAE':>8} | {'R²':>8}")
    print("=" * 80)
    
    for i, feature in enumerate(feature_names):
        y_true_feature = y_true.iloc[:, i] if hasattr(y_true, 'iloc') else y_true[:, i]
        y_pred_feature = y_pred[:, i]
        
        mse = mean_squared_error(y_true_feature, y_pred_feature)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true_feature, y_pred_feature)
        r2 = sklearn_r2_score(y_true_feature, y_pred_feature)
        
        metrics[feature] = {
            'MSE': mse,
            'RMSE': rmse,
            'MAE': mae,
            'R2': r2
        }
        
        print(f"{feature:20} | {mse:8.4f} | {rmse:8.4f} | {mae:8.4f} | {r2:8.4f}")
    
    # 전체 평균 성능
    avg_rmse = np.mean([metric['RMSE'] for metric in metrics.values()])
    avg_mae = np.mean([metric['MAE'] for metric in metrics.values()])
    avg_r2 = np.mean([metric['R2'] for metric in metrics.values()])
    
    print("=" * 80)
    print(f"{'평균':20} | {'':8} | {avg_rmse:8.4f} | {avg_mae:8.4f} | {avg_r2:8.4f}")
    print("=" * 80)
    
    return metrics

# 모델 평가 실행
metrics = evaluate_xgboost_models(y_test, y_pred_combined, models, available_columns)

print(f"\n평가 완료:")
print(f"  - 평가된 지표 수: {len(metrics)}")
print(f"  - 테스트 데이터 크기: {y_test.shape}")
print(f"  - 예측 결과 크기: {y_pred_combined.shape}")

# 모델별 성능 순위
performance_ranking = sorted(metrics.items(), key=lambda x: x[1]['R2'], reverse=True)
print(f"\nR² 기준 성능 순위:")
for i, (feature, metric) in enumerate(performance_ranking, 1):
    print(f"  {i}. {feature}: R² = {metric['R2']:.4f}")

XGBoost 모델 성능 평가
지표명                  |      MSE |     RMSE |      MAE |       R²
leading_index        |   0.0619 |   0.2488 |   0.2204 |  -0.0901
term_spread          |   0.0054 |   0.0734 |   0.0651 |   0.2840
credit_spread        |   0.0299 |   0.1729 |   0.1511 |   0.5541
esi                  |  13.6264 |   3.6914 |   2.9273 |  -1.5001
exchange_usd_krw_close | 19704.5610 | 140.3729 | 133.3171 | -10.0041
cpi_qoq              |   0.1407 |   0.3751 |   0.2967 |  -0.3696
평균                   |          |  24.1558 |  22.8296 |  -1.8543

평가 완료:
  - 평가된 지표 수: 6
  - 테스트 데이터 크기: (7, 6)
  - 예측 결과 크기: (7, 6)

R² 기준 성능 순위:
  1. credit_spread: R² = 0.5541
  2. term_spread: R² = 0.2840
  3. leading_index: R² = -0.0901
  4. cpi_qoq: R² = -0.3696
  5. esi: R² = -1.5001
  6. exchange_usd_krw_close: R² = -10.0041


In [55]:
# XGBoost Feature Importance 분석 (간결화)
def analyze_feature_importance(models, feature_names, top_n=10):
    """
    XGBoost 모델들의 Feature Importance 분석 (시각화 최소화)
    """
    print("XGBoost Feature Importance 분석")
    print("=" * 60)
    
    # 전체 Feature importance 평균 계산
    all_importance = {}
    for feature, model in models.items():
        importance = model.feature_importances_
        for i, feat in enumerate(X_train.columns):
            if feat not in all_importance:
                all_importance[feat] = []
            all_importance[feat].append(importance[i])
    
    # 평균 중요도 계산
    avg_importance = {feat: np.mean(scores) for feat, scores in all_importance.items()}
    top_features = sorted(avg_importance.items(), key=lambda x: x[1], reverse=True)[:top_n]
    
    print(f"상위 {top_n}개 주요 Feature:")
    for i, (feat, importance) in enumerate(top_features, 1):
        print(f"{i:2}. {feat:35}: {importance:.4f}")
    
    # 지표별 top 3 features 요약
    feature_importance_summary = {}
    print(f"\n지표별 Top 3 Features:")
    print("=" * 60)
    
    for feature, model in models.items():
        importance = model.feature_importances_
        feature_df = pd.DataFrame({
            'feature': X_train.columns,
            'importance': importance
        }).sort_values('importance', ascending=False).head(3)
        
        feature_importance_summary[feature] = feature_df['feature'].tolist()
        print(f"\n{feature}:")
        for i, row in feature_df.iterrows():
            print(f"  {row['feature']:30}: {row['importance']:.4f}")
    
    return feature_importance_summary

# Feature importance 분석 실행
feature_importance_summary = analyze_feature_importance(models, available_columns)
print("=" * 60)

XGBoost Feature Importance 분석
상위 10개 주요 Feature:
 1. term_spread_ma_4                   : 0.0991
 2. esi_ma_4                           : 0.0773
 3. exchange_usd_krw_close_ma_4        : 0.0735
 4. cpi_qoq_ma_4                       : 0.0704
 5. credit_spread_lag_1                : 0.0624
 6. esi_lag_2                          : 0.0591
 7. leading_index_qoq                  : 0.0540
 8. term_spread_lag_1                  : 0.0518
 9. esi_lag_1                          : 0.0477
10. term_spread_qoq                    : 0.0443

지표별 Top 3 Features:

leading_index:
  term_spread_ma_4              : 0.3481
  leading_index_lag_1           : 0.0911
  leading_index_ma_4            : 0.0737

term_spread:
  term_spread_ma_4              : 0.1537
  leading_index_qoq             : 0.0927
  esi_ma_4                      : 0.0895

credit_spread:
  esi_lag_2                     : 0.1713
  leading_index_qoq             : 0.1311
  credit_spread_lag_1           : 0.1244

esi:
  esi_ma_4                   

In [56]:
# 2025 Q3 예측 및 최종 결과 정리
def complete_prediction_analysis(models, X_scaled, data, available_columns, metrics):
    """
    2025 Q3 예측 실행 및 최종 결과 정리 (통합 버전)
    """
    print("2025 Q3 XGBoost 예측 및 결과 정리")
    print("=" * 70)
    
    # 1. 2025 Q3 예측 실행
    last_features = X_scaled.iloc[-1].values
    future_predictions = {}
    
    print("1. 2025 Q3 예측 실행...")
    for target in available_columns:
        model = models[target]
        pred = model.predict(last_features.reshape(1, -1))[0]
        future_predictions[target] = pred
    
    # 2. 지표 한글명 매핑
    interpretation = {
        'leading_index': '경기선행지수',
        'term_spread': '장단기금리차', 
        'credit_spread': '신용스프레드',
        'esi': '경제심리지수',
        'exchange_usd_krw_close': '원달러환율',
        'cpi_qoq': '분기물가상승률'
    }
    
    # 3. 예측 결과 출력
    print(f"\n2025년 Q3 경제지표 예측 결과:")
    print("=" * 70)
    print(f"{'지표명':25} | {'예측값':>12} | {'신뢰도':>10}")
    print("=" * 70)
    
    for feature in available_columns:
        value = future_predictions[feature]
        r2 = metrics[feature]['R2']
        confidence = "높음" if r2 > 0.7 else "보통" if r2 > 0.5 else "낮음"
        korean_name = interpretation.get(feature, feature)
        print(f"{korean_name:25} | {value:11.3f} | {confidence:>10}")
    
    # 4. 변화율 분석
    print(f"\n2025 Q2 대비 Q3 변화율:")
    print("=" * 60)
    print(f"{'지표명':25} | {'Q2→Q3':>10} | {'전망':>8}")
    print("=" * 60)
    
    trend_summary = {'상승': 0, '하락': 0}
    if len(data) > 0:
        last_actual = data.iloc[-1]
        for feature in available_columns:
            if feature in last_actual.index:
                q2_value = last_actual[feature]
                q3_value = future_predictions[feature]
                change_rate = ((q3_value - q2_value) / q2_value) * 100
                trend = "상승" if change_rate > 0 else "하락"
                trend_summary[trend] += 1
                
                korean_name = interpretation.get(feature, feature)
                print(f"{korean_name:25} | {change_rate:+9.1f}% | {trend:>8}")
    
    print("=" * 60)
    print(f"상승 예상: {trend_summary['상승']}개, 하락 예상: {trend_summary['하락']}개")
    
    # 5. 성능 요약
    print(f"\n모델 성능 요약:")
    print("=" * 50)
    avg_r2 = np.mean([metrics[f]['R2'] for f in available_columns])
    avg_rmse = np.mean([metrics[f]['RMSE'] for f in available_columns])
    print(f"평균 R² 점수: {avg_r2:.3f}")
    print(f"평균 RMSE: {avg_rmse:.3f}")
    print(f"사용 Feature 수: {X_scaled.shape[1]}")
    print(f"훈련 데이터: {len(X_train)}개 분기")
    
    return future_predictions, trend_summary

# 통합 분석 실행
future_predictions, trend_summary = complete_prediction_analysis(
    models, X_scaled, data, available_columns, metrics
)

# 6. 결과 파일 저장
print(f"\n결과 파일 저장...")
try:
    # 성능 결과
    performance_data = []
    for feature, metric in metrics.items():
        performance_data.append({
            'Feature': feature,
            'RMSE': metric['RMSE'],
            'MAE': metric['MAE'],
            'R2': metric['R2'],
            'Model_Type': 'XGBoost_Optimized'
        })
    performance_df = pd.DataFrame(performance_data).sort_values('R2', ascending=False)
    performance_df.to_csv('xgboost_model_performance.csv', index=False, encoding='utf-8-sig')
    
    # 예측 결과
    interpretation = {
        'leading_index': '경기선행지수', 'term_spread': '장단기금리차', 
        'credit_spread': '신용스프레드', 'esi': '경제심리지수',
        'exchange_usd_krw_close': '원달러환율', 'cpi_qoq': '분기물가상승률'
    }
    
    future_data = []
    for feature in available_columns:
        korean_name = interpretation.get(feature, feature)
        value = future_predictions[feature]
        r2_score = metrics[feature]['R2']
        confidence = "높음" if r2_score > 0.7 else "보통" if r2_score > 0.5 else "낮음"
        
        future_data.append({
            '지표코드': feature,
            '지표명': korean_name,
            '2025_Q3_예측값': round(value, 4),
            'R2_Score': round(r2_score, 4),
            '신뢰도': confidence
        })
    
    future_df = pd.DataFrame(future_data)
    future_df.to_csv('2025_Q3_xgboost_predictions.csv', index=False, encoding='utf-8-sig')
    
    print("결과 파일 저장 완료:")
    print("  - xgboost_model_performance.csv")
    print("  - 2025_Q3_xgboost_predictions.csv")
    
except Exception as e:
    print(f"결과 저장 중 오류: {e}")

print(f"\n" + "=" * 70)
print(f"XGBoost 2025년 Q3 ECOS 지표 예측 분석 완료")
print(f"=" * 70)

2025 Q3 XGBoost 예측 및 결과 정리
1. 2025 Q3 예측 실행...

2025년 Q3 경제지표 예측 결과:
지표명                       |          예측값 |        신뢰도
경기선행지수                    |     100.119 |         낮음
장단기금리차                    |       0.268 |         낮음
신용스프레드                    |       6.183 |         낮음
경제심리지수                    |      88.778 |         낮음
원달러환율                     |    1273.561 |         낮음
분기물가상승률                   |       0.578 |         낮음

2025 Q2 대비 Q3 변화율:
지표명                       |      Q2→Q3 |       전망
경기선행지수                    |      -0.7% |       하락
장단기금리차                    |     -22.8% |       하락
신용스프레드                    |      +7.3% |       상승
경제심리지수                    |      -2.3% |       하락
원달러환율                     |      -9.0% |       하락
분기물가상승률                   |    +128.7% |       상승
상승 예상: 2개, 하락 예상: 4개

모델 성능 요약:
평균 R² 점수: -2.379
평균 RMSE: 18.718
사용 Feature 수: 24
훈련 데이터: 30개 분기

결과 파일 저장...
결과 파일 저장 완료:
  - xgboost_model_performance.csv
  - 2025_Q3_xgboost_predictions.